In [1]:
import os
import re
import csv
import sys
from datetime import datetime
import codecs
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from string import punctuation
from sklearn.model_selection import train_test_split,StratifiedKFold
from sklearn.metrics import roc_auc_score

#from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Flatten, Dense, Input, LSTM, Embedding, Dropout, Activation, SpatialDropout1D
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras.layers import Bidirectional, GlobalMaxPool1D,GlobalMaxPooling1D,GlobalAveragePooling1D ,Conv1D, MaxPooling1D, GRU,CuDNNLSTM,CuDNNGRU, Reshape, MaxPooling1D,AveragePooling1D
from keras.optimizers import RMSprop, SGD
from keras.utils import to_categorical

import colorama
from colorama import Fore


from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'pandas'

In [ ]:
from pathlib import Path
import pandas as pd
from keras.utils import to_categorical

maxlen = 300   # Maximum Sequence Size 
max_features = 250000 # Maximum Number of Words in Dictionary
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1

num_lstm = 300 #300
num_dense = 256 # 256
rate_drop_lstm = 0.05
rate_drop_dense = rate_drop_lstm

loss="val_acc"
#loss="val_loss"
opt='rmsprop'
#opt='adam'

lr=0.01
from keras.optimizers import RMSprop, SGD, Nadam, Adamax, Adam
#opto = SGD(lr=lr, clipvalue=0.5)
#opto=  Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#opto = RMSprop (lr=lr)
#opto = Nadam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
#opto = Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
##
 
act = 'relu'
Trainable=True


data_folder = Path("C:/Users/joshua_giles/Downloads/train")

TRAIN_DATA_FILE = data_folder / "train.csv"

print ("somthing")

secret_saucy_df = pd.read_csv(TRAIN_DATA_FILE)[:176230]  # (176230)
validate_df = pd.read_csv(TRAIN_DATA_FILE)[176231:511230] # (335000)
train_df = pd.read_csv(TRAIN_DATA_FILE)[511231:1672297]

print(train_df.count)
print(validate_df.count)
print(secret_saucy_df.count)

#print(list_sentences_train)
#f = open(file_to_open)

#print(list(train_df))
list_sentences_train = train_df["ReviewText"].fillna("NA").values
list_classes = ["negative", "somewhat negative", "neutral", "somewhat positive", "positive"]
num_classes=5
#y = train_df[list_classes].values
target=train_df['Rating'].values
y1=to_categorical(target)
y=np.delete(y1, 0, axis=1)
list_sentences_test = validate_df["ReviewText"].fillna("NA").values
yaux=y[:,[0]]
#print(f.read())

In [ ]:
def splitstring(s):
    # searching the number of characters to split on
    proposed_pattern = s[0]
    for i, c in enumerate(s[1:], 1):
        if c != " ":
            if proposed_pattern == s[i:(i+len(proposed_pattern))]:
                # found it
                break
            else:
                proposed_pattern += c
    else:
        exit(1)

    return proposed_pattern

In [ ]:
def ReplaceThreeOrMore(s):
    # pattern to look for three or more repetitions of any character, including
    # newlines.
    pattern = re.compile(r"(.)\1{2,}", re.DOTALL) 
    return pattern.sub(r"\1", s)

In [ ]:
import re
def remove_urls (vTEXT):
    vTEXT = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', vTEXT, flags=re.MULTILINE)
    return(vTEXT)

In [ ]:
########################################
## process texts in datasets
########################################

#Regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)

#regex to replace all numerics
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def text_to_wordlist(text,to_lower=False, rem_urls=False, rem_3plus=False, \
                     split_repeated=False, rem_special=False, rep_num=False,
                     man_adj=True, rem_stopwords=False, stem_snowball=False,\
                     stem_porter=False, lemmatize=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    if rem_urls:
        text = remove_urls(text)
    if to_lower:    
        text = text.lower()
    if rem_3plus:    
        text = ReplaceThreeOrMore(text)

    if man_adj: 
        # Clean the text
        text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
        text = re.sub(r"what's", "what is ", text)
        text = re.sub(r"\'s", " ", text)
        text = re.sub(r"\'ve", " have ", text)
        text = re.sub(r"can't", "cannot ", text)
        text = re.sub(r"n't", " not ", text)
        text = re.sub(r"i'm", "i am ", text)
        text = re.sub(r"\'re", " are ", text)
        text = re.sub(r"\'d", " would ", text)
        text = re.sub(r"\'ll", " will ", text)
        text = re.sub(r",", " ", text)
        text = re.sub(r"\.", " ", text)
        text = re.sub(r"!", " ! ", text)
        text = re.sub(r"\/", " ", text)
        text = re.sub(r"\^", " ^ ", text)
        text = re.sub(r"\+", " + ", text)
        text = re.sub(r"\-", " - ", text)
        text = re.sub(r"\=", " = ", text)
        text = re.sub(r"'", " ", text)
        text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
        text = re.sub(r":", " : ", text)
        text = re.sub(r" e g ", " eg ", text)
        text = re.sub(r" b g ", " bg ", text)
        text = re.sub(r" u s ", " american ", text)
        text = re.sub(r"\0s", "0", text)
        text = re.sub(r" 9 11 ", "911", text)
        text = re.sub(r"e - mail", "email", text)
        text = re.sub(r"j k", "jk", text)
        text = re.sub(r"\s{2,}", " ", text)

    # split them into a list
    text = text.split()
    
    if split_repeated:
        for i, c in enumerate(text):
            text[i]=splitstring(c)
    
    # Optionally, remove stop words
    if rem_stopwords:
        stops = set(stopwords.words("english"))
        text = [w for w in text if not w in stops]
    
    text = " ".join(text)
    
    #Remove Special Characters
    if rem_special: 
        text=special_character_removal.sub('',text)
    
    #Replace Numbers
    if rep_num:     
        text=replace_numbers.sub('n',text)

    # Optionally, shorten words to their stems
    if stem_snowball:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    if stem_porter:
        st = PorterStemmer()
        txt = " ".join([st.stem(w) for w in text.split()])
        
    if lemmatize:
        wordnet_lemmatizer = WordNetLemmatizer()
        txt = " ".join([wordnet_lemmatizer.lemmatize(w) for w in text.split()])   
 
    # Return a list of words
    return(text)





In [ ]:
comments = []
for text in list_sentences_train:
    comments.append(text_to_wordlist(text))

In [ ]:
test_comments=[]
for text in list_sentences_test:
    test_comments.append(text_to_wordlist(text))

In [ ]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(comments + test_comments)

sequences = tokenizer.texts_to_sequences(comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)


In [ ]:
print(len(sequences), 'train sequences')
print(len(test_sequences), 'test sequences')
print('Average train sequence length: {}'.format(np.mean(list(map(len, sequences)), dtype=int)))
print('Average test sequence length: {}'.format(np.mean(list(map(len, test_sequences)), dtype=int)))

In [ ]:
print(len(sequences), 'train sequences')
print(len(test_sequences), 'test sequences')
print('Max train sequence length: {}'.format(np.max(list(map(len, sequences)))))
print('Max test sequence length: {}'.format(np.max(list(map(len, test_sequences)))))

In [ ]:
print(len(sequences), 'train sequences')
print(len(test_sequences), 'test sequences')
print('Min train sequence length: {}'.format(np.min(list(map(len, sequences)))))
print('Min test sequence length: {}'.format(np.min(list(map(len, test_sequences)))))

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)

test_data = pad_sequences(test_sequences, maxlen=maxlen)
print('Shape of test_data tensor:', test_data.shape)

In [ ]:
del comments
del test_sequences
del sequences
del list_sentences_train
del list_sentences_test
del train_df
#del test_df

In [ ]:
########################################
## index GLOVE  word vectors
########################################

EMBEDDING_FILE=data_folder  / "glove.840B.300d.txt"
et="GLOVE-840B"

#EMBEDDING_FILE=path+'glove/glove.6B.300d.txt'
#et="GLOVE-6B"

#EMBEDDING_FILE= path+'prodata/toxic_clean_300d.txt'
#et='TOXIC-TXT'

#EMBEDDING_FILE= path+'fasttext/crawl-300d-2M.vec'
#et="FASTTEXT"


print('Indexing '+et+' vectors')
print("Vector",EMBEDDING_FILE )
#Glove Vectors

embeddings_index = {}
f = open(EMBEDDING_FILE,  encoding='utf8')
for line in f:
    values = line.split()
    word = ' '.join(values[:-300])
    coefs = np.asarray(values[-300:], dtype='float32')
    #word = values[0]
    #coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings_index))



In [ ]:
#########
## Glove
#########
all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
emb_mean,emb_std

########################################
## GLOVE prepare embeddings
########################################
print('Preparing embedding matrix')
nb_words = min(max_features, len(word_index))+1
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, EMBEDDING_DIM))
#embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
nl=0
gd=0
for word, i in word_index.items():
    if i >= max_features:
        #print ('Over: ',word)
        nl = nl +1 
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        gd=gd+1
    else:
        #print (word)
        nl = nl +1 

         
del embeddings_index

In [ ]:
#print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0)) 
print( "Matrix", embedding_matrix.shape)
print( "Tamanho Vocabulario", len(word_index), "Maximo de Features",max_features)
print('Null word embeddings: %d' % nl)
print('Good word embeddings: %d' % gd)


In [ ]:
 def create_model0():
    
    mdln="00-max-d-dr-d-dr-d-dr"
    comment_input = Input((maxlen,))

    # we start off with an efficient embedding layer which maps
    # our vocab indices into embedding_dims dimensions
    comment_emb = Embedding(max_features, EMBEDDING_DIM, input_length=maxlen, 
                            embeddings_initializer="uniform")(comment_input)

    # we add a GlobalMaxPooling1D, which will extract features from the embeddings
    # of all words in the comment
    m = GlobalMaxPooling1D()(comment_emb)
    d = Dense(1024, activation=act)(m)
    d = Dropout(rate_drop_dense)(d)
    d = Dense(512, activation=act)(d)
    d = Dropout(rate_drop_dense)(d)
    d = Dense(256, activation=act)(d)
    d = Dropout(rate_drop_dense)(d)
    d = Dense(128, activation=act)(d)
    d = Dropout(rate_drop_dense)(d)
    # We project onto a six-unit output layer, and squash it with a sigmoid:
    output = Dense(num_classes, activation='sigmoid')(d)

    model = Model(inputs=comment_input, outputs=output)

    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    #for layer in model.layers:
     #   weights = layer.get_weights()
    #print(weights)
    
    return model,mdln

In [ ]:
# ## Modelo LSTM Base Line 
########################################
def create_model1():

    mdln="01-lstm-att-d-b"
    embedding_layer = Embedding(nb_words,
            EMBEDDING_DIM,
            weights=[embedding_matrix],
            input_length=maxlen,
            trainable=Trainable)

    lstm_layer = LSTM(num_lstm, dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm,return_sequences=True)

    comment_input = Input(shape=(maxlen,), dtype='int32')
    embedded_sequences= embedding_layer(comment_input)
    x = lstm_layer(embedded_sequences)
    x = Dropout(rate_drop_dense)(x)
    merged = Attention(maxlen)(x)
    merged = Dense(num_dense, activation=act)(merged)
    merged = Dropout(rate_drop_dense)(merged)
    merged = BatchNormalization()(merged)
    preds = Dense(num_classes, activation='sigmoid')(merged)

    model = Model(inputs=[comment_input], \
            outputs=preds)
    model.compile(loss='binary_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])
   
    return model, mdln